# AWS S3 Learning Notebook

## Prerequisites

**Before running this notebook, make sure you're authenticated with AWS SSO:**

```bash
aws sso login --profile cloud-course
```

This command needs to be run in your terminal (not in this notebook) and will:
- Open a browser window for authentication
- Refresh your AWS credentials
- Allow this notebook to interact with AWS services

⚠️ **Note**: If you get a `TokenRetrievalError`, it means your AWS SSO session has expired. Simply run the login command above again.


In [4]:
import boto3
from botocore.exceptions import ClientError, TokenRetrievalError
from uuid import uuid4
from  rich import print
import os
from typing import Optional

try:
    from mypy_boto3_s3 import S3Client
except ImportError:
    print("mypy_boto3_s3 is not installed. Please install it using 'pip install mypy-boto3-s3'")

In [5]:
# Constants

os.environ["AWS_PROFILE"] = "cloud-course"
os.environ["AWS_REGION"] = "us-east-1"

# crearet a session
S3_CLIENT: S3Client = boto3.client("s3")

# create a bucket
BUCKET_NAME = f"cloud-course-bucket-{str(uuid4())[:4]}"

EXAMPLE_OBJECT_KEY = "folder/file.txt"
EXAMPLE_OBJECT_CONTENT = "This is a test file!"

EXAMPLE_OBJECTS = [
    {"example-a/object/file1.txt": "This is a test object."},
    {"example-b/object/file2.txt": "This is another test object."},
    {"example-c/object/file3.txt": "This is a third test object."},
    {"example-d/object/file4.txt": "This is a fourth test object."},
    {"example-e/object/file5.txt": "This is a fifth test object."},
]

print(f"{BUCKET_NAME=}")
print(f"{EXAMPLE_OBJECT_KEY=}")
print(f"{EXAMPLE_OBJECT_CONTENT=}")
print(f"{EXAMPLE_OBJECTS=}")


BUCKET_NAME='cloud-course-bucket-20a2'

EXAMPLE_OBJECT_KEY='folder/file.txt'

EXAMPLE_OBJECT_CONTENT='This is a test file!'

EXAMPLE_OBJECTS=[{'example-a/object/file1.txt': 'This is a test object.'}, {'example-b/object/file2.txt': 'This is 
another test object.'}, {'example-c/object/file3.txt': 'This is a third test object.'}, 
{'example-d/object/file4.txt': 'This is a fourth test object.'}, {'example-e/object/file5.txt': 'This is a fifth 
test object.'}]

In [6]:
# Create a bucket

try:
    from mypy_boto3_s3.type_defs import CreateBucketOutputTypeDef
except ImportError:
    print("mypy-boto3-s3 is not installed, skipping type checking")


def create_bucket(bucket_name: str) -> Optional["CreateBucketOutputTypeDef"]:
    """
    Create a bucket
    
    Args:
        bucket_name: The name of the bucket to create

    Returns:
        The output of the create_bucket operation
    """
    try:
        return S3_CLIENT.create_bucket(Bucket=bucket_name)
    except TokenRetrievalError:
        print("❌ AWS SSO token has expired!")
        print("🔧 Please run this command in your terminal:")
        print("   aws sso login --profile <profile_name>")
        print("📝 Then restart this notebook and try again.")
        raise
    except ClientError as e:
        error_code = e.response.get("Error", {}).get("Code")
        if error_code == 'BucketAlreadyOwnedByYou':
            print(f"✅ Bucket {bucket_name} already exists and is owned by you")
            return None
        elif error_code == 'BucketAlreadyExists':
            print(f"❌ Bucket {bucket_name} already exists and is owned by someone else")
            raise
        else:
            print(f"❌ Error creating bucket: {e}")
            raise


response = create_bucket(bucket_name=BUCKET_NAME)
if response:
    print(f"✅ Bucket {BUCKET_NAME} created successfully")
    print(f"{response=}")
else:
    print(f"ℹ️  Using existing bucket {BUCKET_NAME}")


✅ Bucket cloud-course-bucket-20a2 created successfully

response={'ResponseMetadata': {'RequestId': 'A89Z1JWPMQQPTKZA', 'HostId': 
'sxhBQKBp3SLFztyQ7uheU3uzwaRCxsY7CuNGBTkhLcNESbarsKqrrBtS5SMbhczlNPjWuSX69++04SqiW15c8sixruq5l66xX4BFc/SRd/A=', 
'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 
'sxhBQKBp3SLFztyQ7uheU3uzwaRCxsY7CuNGBTkhLcNESbarsKqrrBtS5SMbhczlNPjWuSX69++04SqiW15c8sixruq5l66xX4BFc/SRd/A=', 
'x-amz-request-id': 'A89Z1JWPMQQPTKZA', 'date': 'Fri, 20 Jun 2025 01:40:52 GMT', 'location': 
'/cloud-course-bucket-20a2', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Location': 
'/cloud-course-bucket-20a2'}

In [7]:
# write an object to the bucket
try:
    from mypy_boto3_s3.type_defs import PutObjectOutputTypeDef
except ImportError:
    print("mypy-boto3-s3 is not installed, skipping type checking")


def write_text_object_to_bucket(
    bucket_name: str, 
    object_key: str, 
    object_content: str) -> Optional["PutObjectOutputTypeDef"]:
    """
    Write a text object to the bucket
    """
    try:
        return S3_CLIENT.put_object(Bucket=bucket_name, Key=object_key, Body=object_content)
    except ClientError as e:
        error_code = e.response.get("Error", {}).get("Code")
        raise Exception(f"❌ Error writing object to bucket: {error_code}")


# write a single example object to the bucket
response = write_text_object_to_bucket(
    bucket_name=BUCKET_NAME, 
    object_key=EXAMPLE_OBJECT_KEY, 
    object_content=EXAMPLE_OBJECT_CONTENT)

print(f"✅ Object written to path 's3://{BUCKET_NAME}/{EXAMPLE_OBJECT_KEY}'")
print(f"{response=}")


✅ Object written to path 's3://cloud-course-bucket-20a2/folder/file.txt'

response={'ResponseMetadata': {'RequestId': 'A89PPV2MDKWK7RS3', 'HostId': 
'2Bq6RI+14lp9SNpYdzI5/29RUHtdMid2nhIMDk3POTEofd4/HA8W/7qhINv3Dv1m6GUYL9NM19/ssgcb0PtgJO9s4nH8P527G43lKZ7vQog=', 
'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 
'2Bq6RI+14lp9SNpYdzI5/29RUHtdMid2nhIMDk3POTEofd4/HA8W/7qhINv3Dv1m6GUYL9NM19/ssgcb0PtgJO9s4nH8P527G43lKZ7vQog=', 
'x-amz-request-id': 'A89PPV2MDKWK7RS3', 'date': 'Fri, 20 Jun 2025 01:40:52 GMT', 'x-amz-server-side-encryption': 
'AES256', 'etag': '"07aa24d4ee5d5d29fbaf360896713947"', 'x-amz-checksum-crc32': 'fNXj1g==', 'x-amz-checksum-type': 
'FULL_OBJECT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': 
'"07aa24d4ee5d5d29fbaf360896713947"', 'ChecksumCRC32': 'fNXj1g==', 'ChecksumType': 'FULL_OBJECT', 
'ServerSideEncryption': 'AES256'}

In [8]:
# write multiple example objects to the bucket
for example_object in EXAMPLE_OBJECTS:
    for object_key, object_content in example_object.items():
        print(f"Writing object to path 's3://{BUCKET_NAME}/{object_key}'")
        write_text_object_to_bucket(
            bucket_name=BUCKET_NAME, 
            object_key=object_key, 
            object_content=object_content
            )

Writing object to path 's3://cloud-course-bucket-20a2/example-a/object/file1.txt'

Writing object to path 's3://cloud-course-bucket-20a2/example-b/object/file2.txt'

Writing object to path 's3://cloud-course-bucket-20a2/example-c/object/file3.txt'

Writing object to path 's3://cloud-course-bucket-20a2/example-d/object/file4.txt'

Writing object to path 's3://cloud-course-bucket-20a2/example-e/object/file5.txt'

In [9]:
# read object from s3
from botocore.response import StreamingBody

def read_text_object_from_bucket(
    bucket_name: str, 
    object_key: str) -> str | None:
    """
    Read a text object from the bucket
    """
    try:
        response = S3_CLIENT.get_object(Bucket=bucket_name, Key=object_key)
        content_streaming_body: StreamingBody = response["Body"]
        content:str = content_streaming_body.read().decode("utf-8")
        return content
    except ClientError as e:
        raise Exception(f"❌ Error reading object from bucket: {e}")

In [10]:
# read a single example object from the bucket
content = read_text_object_from_bucket(
    bucket_name=BUCKET_NAME, 
    object_key=EXAMPLE_OBJECT_KEY
    )

if content:
    print(f"✅ Object read from path 's3://{BUCKET_NAME}/{EXAMPLE_OBJECT_KEY}'")
    print(f"{content=}")
else:
    print(f"❌ Object not found in path 's3://{BUCKET_NAME}/{EXAMPLE_OBJECT_KEY}'")

✅ Object read from path 's3://cloud-course-bucket-20a2/folder/file.txt'

content='This is a test file!'

In [ ]:
# list objects in the bucket
try:
    from mypy_boto3_s3.type_defs import ListObjectsV2OutputTypeDef
except ImportError:
    print("mypy-boto3-s3 is not installed, skipping type checking")

def list_all_object_keys_in_bucket(
    bucket_name: str
    ) -> list[str] | None:
    """
    List objects in the bucket
    """
    try:
        response = S3_CLIENT.list_objects_v2(Bucket=bucket_name)
        return response["Contents"]
    except ClientError as e:

In [ ]:
# List all objects in the bucket
objects = list_all_object_keys_in_bucket(BUCKET_NAME)
if objects:
    print("Objects in bucket:")
    for obj in objects:
        print(f" - {obj}")